# Day 7: AI-Assisted IaC Code Review - Enhanced Workflow

This notebook demonstrates a workflow for reviewing Terraform IaC files using both static analysis tools and a Large Language Model (LLM). Outputs are saved and compared to highlight the strengths of each approach.

## 1. Run Static Analysis with tfsec and checkov

The following cells will run tfsec and checkov on all Terraform files in the project and save their outputs to markdown files.

In [1]:
import os

# Find all .tf files in the workspace
terraform_files = []
for root, dirs, files in os.walk('../../..'):
    for file in files:
        if file.endswith('.tf'):
            terraform_files.append(os.path.join(root, file))

print(f"Found {len(terraform_files)} Terraform files:")
for tf_file in terraform_files:
    print(tf_file)

Found 4 Terraform files:
../../..\IT\AWS Architecture using Terraform\provider.tf
../../..\IT\AWS Architecture using Terraform\sg_asg_alb.tf
../../..\IT\AWS Architecture using Terraform\vpc.tf
../../..\IT\Cloud-Resource-Optimization-Tool\terraform\main.tf


In [2]:
import subprocess
import os

output_dir = os.path.join(os.getcwd(), "Iac analysis")
os.makedirs(output_dir, exist_ok=True)

for tf_file in terraform_files:
    base_name = os.path.basename(tf_file)
    tfsec_dir = os.path.dirname(tf_file)
    output_file = os.path.join(output_dir, base_name + '_tfsec.md')
    try:
        result = subprocess.run(["tfsec", tfsec_dir], capture_output=True, text=True)
        output = result.stdout
    except FileNotFoundError:
        output = "tfsec not installed or not found in PATH. Please install tfsec to run static analysis."
    except Exception as e:
        output = f"tfsec error: {e}"
    with open(output_file, "w") as f:
        f.write(f"# tfsec Analysis for {tf_file}\n\n")
        f.write(output)
    print(f"tfsec output saved to {output_file}")

tfsec output saved to c:\Users\behip\Documents\Work\IT\30-Days-of-AI-in-Devops-SRE-Challenge-\Day 7 - AI-Assisted IaC Code Review\Iac analysis\provider.tf_tfsec.md
tfsec output saved to c:\Users\behip\Documents\Work\IT\30-Days-of-AI-in-Devops-SRE-Challenge-\Day 7 - AI-Assisted IaC Code Review\Iac analysis\sg_asg_alb.tf_tfsec.md
tfsec output saved to c:\Users\behip\Documents\Work\IT\30-Days-of-AI-in-Devops-SRE-Challenge-\Day 7 - AI-Assisted IaC Code Review\Iac analysis\sg_asg_alb.tf_tfsec.md
tfsec output saved to c:\Users\behip\Documents\Work\IT\30-Days-of-AI-in-Devops-SRE-Challenge-\Day 7 - AI-Assisted IaC Code Review\Iac analysis\vpc.tf_tfsec.md
tfsec output saved to c:\Users\behip\Documents\Work\IT\30-Days-of-AI-in-Devops-SRE-Challenge-\Day 7 - AI-Assisted IaC Code Review\Iac analysis\vpc.tf_tfsec.md
tfsec output saved to c:\Users\behip\Documents\Work\IT\30-Days-of-AI-in-Devops-SRE-Challenge-\Day 7 - AI-Assisted IaC Code Review\Iac analysis\main.tf_tfsec.md
tfsec output saved to c:\U

## 2. LLM Review of Terraform Files (Ollama)

The following cell sends each Terraform file to an LLM (Ollama) for review and saves the feedback to markdown files.

In [3]:
# TEMPORARY: Only analyze vpc.tf for faster testing
# Remove or comment out this cell to analyze all Terraform files
terraform_files = [tf for tf in terraform_files if os.path.basename(tf) == 'vpc.tf']
print(f"Analyzing only: {terraform_files}")

Analyzing only: ['../../..\\IT\\AWS Architecture using Terraform\\vpc.tf']


In [7]:
import requests
import os

OLLAMA_URL = "http://localhost:11434/api/generate"
OLLAMA_MODEL = "codellama"  # Change to your preferred model

review_prompt_template = (
    "You are a senior DevOps engineer and security reviewer. "
    "Carefully review the following Terraform code for security vulnerabilities, misconfigurations, performance issues, compliance gaps, and best-practice violations. "
    "Provide actionable feedback, specific recommendations, and highlight any critical, high, medium, or low severity issues you find in the code itself. "
    "Organize your feedback into sections: Security, Performance, Compliance, and Best Practices. "
    "For each issue, provide a clear explanation, severity, and actionable improvement.\n\n"
    "Terraform code:\n{terraform_code}\n"
)

output_dir = os.path.join(os.getcwd(), "Iac analysis")
os.makedirs(output_dir, exist_ok=True)

for tf_file in terraform_files:
    base_name = os.path.basename(tf_file)
    with open(tf_file, "r") as f:
        terraform_code = f.read()
    review_prompt = review_prompt_template.format(terraform_code=terraform_code)
    payload = {
        "model": OLLAMA_MODEL,
        "prompt": review_prompt,
        "stream": False
    }
    response = requests.post(OLLAMA_URL, json=payload)
    feedback = response.json().get("response", "No feedback received.")
    output_file = os.path.join(output_dir, base_name + "_ollama_review.md")
    with open(output_file, "w") as f:
        f.write(f"# Ollama LLM Review for {tf_file}\n\n")
        f.write(feedback)
    print(f"Ollama review saved to {output_file}")

Ollama review saved to c:\Users\behip\Documents\Work\IT\30-Days-of-AI-in-Devops-SRE-Challenge-\Day 7 - AI-Assisted IaC Code Review\Iac analysis\vpc.tf_ollama_review.md


## 3. Comparison: tfsec vs Ollama LLM Review for vpc.tf

### tfsec Output Highlights
- tfsec provides automated static analysis, flagging security vulnerabilities, misconfigurations, and best-practice violations based on predefined rules.
- The output is structured, listing passed checks and detected issues with severity levels and recommendations.
- tfsec excels at quickly identifying known patterns and compliance gaps, but may miss context-specific or nuanced issues.

### Ollama LLM Output Highlights
- Ollama (LLM) reviews the Terraform code directly, offering human-like feedback and deeper reasoning.
- The LLM can identify issues missed by static analysis, provide context-aware suggestions, and explain the rationale behind recommendations.
- Feedback is organized into sections (Security, Performance, Compliance, Best Practices) and is more actionable and readable.
- Ollama is used in this project because it is free and runs locally, making it accessible for experimentation and learning.
- For even better results, one can use commercial LLMs such as OpenAI's GPT models via API keys, which may provide more advanced analysis and richer feedback.

### Key Differences
- tfsec is rule-based and deterministic; Ollama is context-aware and can reason about code beyond static rules.
- Ollama may catch subtle or emerging risks, while tfsec is limited to its rule set.
- Combining both approaches provides broader coverage: tfsec for quick checks, Ollama for deeper review.

### Conclusion
Using both tfsec and Ollama LLM for IaC code review offers complementary strengths. tfsec ensures baseline security and compliance, while Ollama provides expert-level, context-sensitive feedback. For best results, use tfsec to catch common issues and LLMs to uncover complex, nuanced, or human-centric risks.